In [5]:
import pandas as pd
df=pd.read_csv('/content/tictac_single.txt',delimiter=" ",header=None)

In [6]:
df.head()

,0,1,2,3,4,5,6,7,8,9
0,1,-1,0,0,0,0,0,1,0,6
1,1,0,1,0,0,0,0,-1,0,1
2,0,0,0,0,0,1,0,0,0,2
3,0,1,1,0,0,0,0,-1,0,0
4,1,1,-1,0,0,0,0,-1,0,3


In [13]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1]


In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=43)

In [15]:
X_train.shape

(4913, 9)

In [19]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier().fit(X_train,y_train)
y_predict=model.predict(X_test)


In [20]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_predict)

0.8943833943833944

In [22]:
model.predict([[1,0,0,0,0,0,0,0,0]])

array([4])

In [27]:
class Board:
    def __init__(self):
        self.board = [[" " for _ in range(3)] for _ in range(3)]

    def printBoard(self):
        print("-" * 17)
        print("|R\\C| 0 | 1 | 2 |")
        print("-" * 17)
        for i in range(3):
            print(f"| {i} | {self.board[i][0]} | {self.board[i][1]} | {self.board[i][2]} |")
            print("-" * 17)

    def to_model_input(self):
        """Convert board to a flat list format needed by the model."""
        model_input = []
        for row in self.board:
            for cell in row:
                if cell == "X":
                    model_input.append(1)
                elif cell == "O":
                    model_input.append(-1)
                else:
                    model_input.append(0)
        return model_input

    def place_move(self, row, col, player):
        self.board[row][col] = player

class Game:
    def __init__(self, model):
        self.board = Board()
        self.turn = 'X'
        self.turns = 0
        self.model = model

    def switchPlayer(self):
        self.turn = 'O' if self.turn == 'X' else 'X'

    def validateEntry(self, row, col):
        if not (0 <= row < 3 and 0 <= col < 3):
            print("Invalid entry: try again.")
            print("Row & column numbers must be either 0, 1, or 2.")
            return False
        elif self.board.board[row][col] != " ":
            print("That cell is already taken.")
            print("Please make another selection.")
            return False
        return True

    def checkFull(self):
        return all(cell != " " for row in self.board.board for cell in row)

    def checkWin(self):
        b = self.board.board
        player = self.turn
        for i in range(3):
            if b[i][0] == b[i][1] == b[i][2] == player or b[0][i] == b[1][i] == b[2][i] == player:
                return True
        if b[0][0] == b[1][1] == b[2][2] == player or b[0][2] == b[1][1] == b[2][0] == player:
            return True
        return False

    def checkEnd(self):
        if self.checkWin():
            print(f"{self.turn} IS THE WINNER!!!")
            return True
        elif self.checkFull():
            print("DRAW! NOBODY WINS!")
            return True
        return False

    def ai_move(self):

      model_input = self.board.to_model_input()
      move = self.model.predict([model_input])[0]  # model expects a 2D array, so wrap in another list
      move -= 1  # Convert 1-9 to 0-8 index
      row, col = divmod(move, 3)

      # Check if the chosen cell is already occupied
      if self.board.board[row][col] != " ":
          # Find first available move
          for i in range(9):
              row, col = divmod(i, 3)
              if self.board.board[row][col] == " ":
                  break
      return row, col

    def playGame(self):
        repeat = "y"
        while repeat[0].lower() == "y":
            self.board = Board()
            self.turns = 0
            self.turn = 'X'
            print("New Game: X goes first.")
            self.board.printBoard()

            while True:
                print(f"\n{self.turn}'s turn.")

                if self.turn == 'X':
                    row, col = map(int, input("Enter row and column (e.g., 0,1): ").split(","))
                    if not self.validateEntry(row, col):
                        continue
                else:
                    row, col = self.ai_move()
                    print(f"AI selects row {row}, column {col}")

                self.board.place_move(row, col, self.turn)
                self.turns += 1

                if self.checkEnd():
                    self.board.printBoard()
                    break

                self.board.printBoard()
                self.switchPlayer()

            print()
            repeat = input("Another game? Enter Y or y for yes.\n")
        print("Thank you for playing")

if __name__ == "__main__":

  game = Game(model)
  game.playGame()



New Game: X goes first.
-----------------
|R\C| 0 | 1 | 2 |
-----------------
| 0 |   |   |   |
-----------------
| 1 |   |   |   |
-----------------
| 2 |   |   |   |
-----------------

X's turn.
Enter row and column (e.g., 0,1): 0,0
-----------------
|R\C| 0 | 1 | 2 |
-----------------
| 0 | X |   |   |
-----------------
| 1 |   |   |   |
-----------------
| 2 |   |   |   |
-----------------

O's turn.
AI selects row 1, column 0
-----------------
|R\C| 0 | 1 | 2 |
-----------------
| 0 | X |   |   |
-----------------
| 1 | O |   |   |
-----------------
| 2 |   |   |   |
-----------------

X's turn.
Enter row and column (e.g., 0,1): 0,1
-----------------
|R\C| 0 | 1 | 2 |
-----------------
| 0 | X | X |   |
-----------------
| 1 | O |   |   |
-----------------
| 2 |   |   |   |
-----------------

O's turn.
AI selects row 0, column 2
-----------------
|R\C| 0 | 1 | 2 |
-----------------
| 0 | X | X | O |
-----------------
| 1 | O |   |   |
-----------------
| 2 |   |   |   |
----------